# Linear Unmixing for Sentinel-2 Satellite Imagery

Michael Mommert, Stuttgart University of Applied Sciences, 2025

This Notebook introduces the concepts of linear unmixing for multispectral Sentinel-2 imaging data. Using a very simple example, we showcase an unmixing method for three distinct materials based on least-squares fitting.

In [ ]:
import os
import zipfile

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
from scipy.optimize import fmin_slsqp

## Data Download

We will download a small sample dataset containing Sentinel-2 satellite imagery and unpack the dataset:

In [ ]:
# download dataset
!wget https://zenodo.org/records/12819787/files/sentinel2_coastal_scenes.zip?download=1 -O sentinel2.zip

import zipfile

# extract dataset zipfile
with zipfile.ZipFile('sentinel2.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


We read in all images from the dataset into a single NumPy array:

In [ ]:
data = []
for filename in sorted(os.listdir('data/')):
    if filename.endswith('.npy'):
        data.append(np.load(open(os.path.join('data', filename), 'rb'), allow_pickle=True))
data = np.array(data)

The `data` array is built up in such a way that it contains 5 images, 12 bands per image and each image has a height of 120 pixels and a width of 120 pixels. This dimensionality is reflected by the shape of the array:

In [ ]:
data.shape

Let's plot one of the images.

In [ ]:
i = 3  # image id

# first, we extract the R, G and B bands and stack them into the shape [120, 120, 3]
img = np.dstack([data[i][3], data[i][2], data[i][1]])

# then we normalize the pixel values in such a way that they range from 0 (min) to 1 (max)
img = (img-np.min(img, axis=(0,1)))/(np.max(img, axis=(0,1)) - np.min(img, axis=(0,1)))

# now we can plot the image
plt.imshow(img)

We pick a pixel in that area of the image that is seemingly covered by grass and plot its spectral distribution. Note that the coordinate notation is [row, column].

In [ ]:
# define band names and central wavelengths
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
band_lambdas = [442.7, 492.4, 559.8, 664.6, 704.1, 740.5, 782.8, 832.8, 864.7, 945.1, 1613.7, 2202.4]

sample = data[3, :, 70, 40]/10000 # divide by 10k to produce spectral reflectance

plt.plot(bands, sample)
plt.xlabel('Band')
plt.ylabel('Spectral Reflectance')

Naturally, this plot will look different if we plot the spectral reflectance as a function of the bands' central wavelengths.

In [ ]:
plt.plot(band_lambdas, sample)
plt.xlabel('Wavelength (micron)')
plt.ylabel('Spectral Reflectance')

We define some reference materials against which we can compare this spectrum: grass, soil and concrete).

In [ ]:
# matrix containing reference spectral reflectances of grass, soil and concrete (columns, from Copernicus Browser)
A = np.array(
    [[0.0406, 0.0522, 0.2932],
	 [0.0434, 0.0847, 0.3085],
	 [0.1097, 0.1496, 0.3371],
	 [0.0451, 0.2154, 0.3469],
	 [0.1720, 0.2328, 0.3498],
	 [0.3990, 0.2480, 0.3528],
	 [0.4765, 0.2612, 0.3538],
	 [0.4937, 0.2737, 0.3478],
	 [0.5042, 0.2801, 0.3419],
	 [0.4974, 0.3015, 0.3366],
	 [0.3067, 0.4020, 0.3040],
	 [0.1709, 0.4028, 0.2221]])

Now we can compare our pixel spectrum to these reference spectra.

In [ ]:
plt.plot(band_lambdas, sample, label='sample')
plt.xlabel('Wavelength (micron)')
plt.ylabel('Spectral Reflectance')

plt.plot(band_lambdas, A[:, 0], color='green', label='grass')
plt.plot(band_lambdas, A[:, 1], color='brown', label='soil')
plt.plot(band_lambdas, A[:, 2], color='gray', label='concrete')
plt.legend()

Our sample clearly shows the characteristic red edge behavior that is indicative of vegetation. But the spectral reflectance is significantly lower. **Why?**

**Exercise**: Scale the grass reference spectrum by a constant factor to make it fit the sample spectrum by eye. The factor tells you how much lower the reflectance of our sample is compared to the reference spectrum of grass.

In [ ]:
# use this cell for the exercise

## Linear Unmixing

We apply linear unmixing to reproduce the spectral signature of our sample.

First, create a function that outputs the modeled spectral signature based on an input vector x that reflects the abundances of the three considered materials: grass, soil and concrete.

In [ ]:
def model(x):
    """Output model spectrum given x."""
    return np.matmul(A, x)

For a given abundance vector, we obtain a model spectrum.

In [ ]:
model([0, 1, 0])

Let's plot this spectrum.

In [ ]:
plt.plot(band_lambdas, sample, label='sample')
plt.xlabel('Wavelength (micron)')
plt.ylabel('Spectral Reflectance')

plt.plot(band_lambdas, model([0, 1, 0]), color='red', label='model')
plt.legend()

There is not much resemblance of our sample and a model that contains only soil...

**Exercise**: Reuse the previous cell and modify the abundance vector manually to fit a suitable model to our sample pixel. What does x look like?

In [ ]:
# use this cell for the exercise

Now we will take a more systematic approach and use a least-squares fitting method to find the best-fit x. 

What least-squares fitting does is to minimize the squared errors between our model and the measured sample spectrum. For this purpose, we use a fitting routine, which aims to minimize a function based on a number of function parameters (in our case: the abundance vector). Once a set of suitable abundances are found that minimize the squared errors, this vector is output.

Our `model` function currently only outputs a model spectrum based on a given abundance vector. Therefore, we need a second function, which we will call `squared_residuals` that will output the sum over the squared residuals between our sample spectrum and the model spectrum based on an abundance vector. It is this function that we will minimize in the following.

In [ ]:
def squared_residuals(x):
    """Compute and sum up squared residuals between `sample` and `model(x)`."""
    return np.sum((model(x)-sample)**2)

res = fmin_slsqp(squared_residuals,  # the function which we will minimize (note that we don't pass any arguments to this function)
                 [0, 0, 0],  # an initial guess for our abundace vector (could be anything)
                 bounds=((0, 1), (0, 1), (0, 1)))  # the bounding conditions: each element of x must have a value between 0 and 1
res

The values of the resulting abundance vector are shown above. We see that the abundance of grass is high (58%), but the abundaces of soil and concrete are zero. Is it a problem that the sum of all abundances is less than unity? Not necessarily. Think of it in the following way: our fitting process finds a 100% abundance of grass, but no soil or concrete in this pixel. We can interpret the 58% value as a modulation of the overall reflectance of the materials in this pixel (the grass in this pixel might be of a different species that has as 58% lower reflectance than the grass that was used in the reference spectrum.) 

Let's plot the model spectrum and compare it to the sample spectrum.

In [ ]:
plt.plot(band_lambdas, sample, label='sample')
plt.xlabel('Wavelength (micron)')
plt.ylabel('Spectral Reflectance')

plt.plot(band_lambdas, model(res), color='red', label='model')
plt.legend()

This looks good. The agreement of the model and sample spectra is good, especially at short wavelengths. The discrepancy at longer wavelengths is clear, but may simply be related to different plant species. 

**Exercise**: Pick a sand sample pixel and model its spectral properties using the same method. What are the spectral abundances?

In [ ]:
# use this cell for the exercise

**Exercise**: Consider the following sample pixel and derive the spectral abundances. How do you interpret this information with respect to the underlying land cover?

In [ ]:
sample = data[3, :, 83, 36]/10000 # divide by 10k to produce spectral reflectance

# use this cell for the exercise

So far, we have only considered single sample pixels. Let's go one step further an derive the mean spectral properties for an area. We pick an area of grassland and average the pixel values across the spectral bands.

In [ ]:
sample = np.average(data[3, :, 70:80, 40:60], axis=(1,2))/10000
sample

Naturally, we can model the spectral abundances across this area in exactly the same way that we did for a single pixel:

In [ ]:

res = fmin_slsqp(squared_residuals, 
                 [0, 0, 0],
                 bounds=((0, 1), (0, 1), (0, 1)))
print('abundances', res)

plt.plot(band_lambdas, sample, label='sample')
plt.xlabel('Wavelength (micron)')
plt.ylabel('Spectral Reflectance')

plt.plot(band_lambdas, model(res), color='red', label='model')
plt.legend()

As you can see, the model fits the data extremely well for the short wavelengths. This may well be since we are averaging over a large number of pixels (and spectra).